In [1]:
import os
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from torchvision.models import resnet34, ResNet34_Weights
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize, Grayscale

from zennit.attribution import Gradient, SmoothGrad
from zennit.torchvision import ResNetCanonizer
from zennit.composites import EpsilonGammaBox, EpsilonPlusFlat, EpsilonAlpha2Beta1Flat, EpsilonAlpha2Beta1, EpsilonPlus
from zennit.image import imgify, imsave

from PIL import Image

from tempo.models import NewTempoLinear
from tempo.data.datasets import finetune_dataset2

In [2]:
MODEL_NAME = "test_18.pth"

In [3]:
# load models
weights_tp = torch.load(f'../model_zoo/{MODEL_NAME}')
model_tp = NewTempoLinear(out_features=24, weights=None,freeze_backbone=True)
model_tp.load_state_dict(weights_tp)
model_tp.requires_grad = True
_ = model_tp.eval()

weights_bl = torch.load('../model_zoo/baseline.pth')
model_bl = NewTempoLinear(out_features=24, weights=None,freeze_backbone=True)
model_bl.load_state_dict(weights_bl)
model_bl.requires_grad = True
_ = model_bl.eval()

In [4]:
def get_relevance(model, data, num_classes):

    with torch.no_grad():
        pred = torch.argmax(model(data).flatten()).item()

    canonizer = ResNetCanonizer()

    # create a composite, specifying the canonizers
    composite = EpsilonPlusFlat(canonizers=[canonizer])

    # choose a target class for the attribution (label 437 is lighthouse)
    target = torch.eye(num_classes)[[pred]]

    # create the attributor, specifying model and composite
    with Gradient(model=model, composite=composite) as attributor:
        # compute the model output and attribution
        output, attribution = attributor(data, target)

    relevance = attribution.sum(1)

    return pred, relevance

In [5]:
# Load test-set
test_loader = finetune_dataset2(train=False, batch_size=1)

In [6]:
normalize = Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # imagenet normalization, also applied during tempo training

In [7]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [8]:
# fig = plt.figure(figsize=(200., 200.))
# grid = ImageGrid(fig, 111,
#                  nrows_ncols=(test_loader.dataset.__len__(), 3),
#                  axes_pad=0.5,
#                  )

# grid = chunker(grid, 3)
# for ax, (im, lbl) in zip(grid, test_loader):
#     img_n = normalize(im)
#     pred, relevance = get_relevance(model_tp, img_n, 24)
#     pred_bl, relevance_bl = get_relevance(model_bl, img_n, 24)
#     img_hm = imgify(relevance, symmetric=True, cmap='coldnhot')
#     img_hm_bl = imgify(relevance_bl, symmetric=True, cmap='coldnhot')

#     ax[0].set_title(f'Target: {lbl.item()}')
#     ax[1].set_title(f'Prediction: {pred}')
#     ax[2].set_title(f'Prediction: {pred_bl}')

#     im = im[0]
#     ax[0].imshow(im.permute(1,2,0))
#     ax[1].imshow(img_hm)
#     ax[2].imshow(img_hm_bl)

# plt.show()

In [9]:
class_map = {
            'A': 0, 
            'B': 1, 
            'C': 2,
            'D': 3,
            'E': 4,
            'F': 5,
            'G': 6,
            'H': 7,
            'I': 8,
            'K': 9,
            'L': 10,
            'M': 11,
            'N': 12,
            'O': 13,
            'P': 14,
            'Q': 15,
            'R': 16,
            'S': 17,
            'T': 18,
            'U': 19,
            'V': 20,
            'W': 21,
            'X': 22,
            'Y': 23,
            }

inv_class_map = {v: k for k, v in class_map.items()}

In [10]:
def get_relevance_new(model, data, num_classes):

    with torch.no_grad():
        _, (idx1, idx2, idx3) = torch.topk(model(data).flatten(), 3)

    canonizer = ResNetCanonizer()

    # low, high = normalize(torch.tensor([[[[[0.]]] * 3], [[[[1.]]] * 3]]))
    # composite = EpsilonGammaBox(low=low, high=high, canonizers=[canonizer])

    # create a composite, specifying the canonizers
    composite = EpsilonPlusFlat(canonizers=[canonizer])

    # composite = EpsilonAlpha2Beta1(canonizers=[canonizer])
    # composite = EpsilonAlpha2Beta1Flat(canonizers=[canonizer])
    


    # choose a target class for the attribution
    target1 = torch.eye(num_classes)[[idx1.item()]]
    # target2 = torch.eye(num_classes)[[idx2.item()]]
    # target3 = torch.eye(num_classes)[[idx3.item()]]

    # create the attributor, specifying model and composite
    with Gradient(model=model, composite=composite) as attributor:
        # compute the model output and attribution
        output1, attribution1 = attributor(data, target1)
        # output2, attribution2 = attributor(data, target2)
        # output3, attribution3 = attributor(data, target3)

    relevance1 = attribution1.sum(1)
    # relevance2 = attribution2.sum(1)
    # relevance3 = attribution3.sum(1)

    return (idx1, relevance1)#, (idx2, relevance2), (idx3, relevance3)

In [11]:
fig = plt.figure(figsize=(250., 250.))
grid = ImageGrid(fig, 111,
                 nrows_ncols=(test_loader.dataset.__len__(), 3),
                 axes_pad=0.5,
                 )

grid = chunker(grid, 3)
for ax, (im, lbl) in zip(grid, test_loader):
    img_n = normalize(im)
    # (pred1, relevance1), (pred2, relevance2), (pred3, relevance3) = get_relevance_new(model_tp, img_n, 24)
    # (pred_bl_1, relevance_bl_1), (pred_bl_2, relevance_bl_2), (pred_bl_3, relevance_bl_3) = get_relevance_new(model_bl, img_n, 24)
    pred1, relevance1 = get_relevance_new(model_tp, img_n, 24)
    pred_bl_1, relevance_bl_1 = get_relevance_new(model_bl, img_n, 24)
    


    img_hm_1 = imgify(relevance1, symmetric=True, cmap='coldnhot')
    # img_hm_2 = imgify(relevance2, symmetric=True, cmap='coldnhot')
    # img_hm_3 = imgify(relevance3, symmetric=True, cmap='coldnhot')

    img_hm_bl_1 = imgify(relevance_bl_1, symmetric=True, cmap='coldnhot')
    # img_hm_bl_2 = imgify(relevance_bl_2, symmetric=True, cmap='coldnhot')
    # img_hm_bl_3 = imgify(relevance_bl_3, symmetric=True, cmap='coldnhot')

    ax[0].set_title(f'Target: {inv_class_map[lbl.item()]}')
    ax[1].set_title(f'Tempo: {inv_class_map[pred1.item()]}')
    ax[2].set_title(f'Baseline: {inv_class_map[pred_bl_1.item()]}')
    # ax[1].set_title(f'Prediction: {inv_class_map[pred1.item()]}')
    # ax[2].set_title(f'Prediction: {inv_class_map[pred2.item()]}')
    # ax[3].set_title(f'Prediction: {inv_class_map[pred3.item()]}')
    # ax[4].set_title(f'Prediction: {inv_class_map[pred_bl_1.item()]}')
    # ax[5].set_title(f'Prediction: {inv_class_map[pred_bl_2.item()]}')
    # ax[6].set_title(f'Prediction: {inv_class_map[pred_bl_3.item()]}')

    im = im[0]
    ax[0].imshow(im.permute(1,2,0))
    ax[1].imshow(img_hm_1)
    ax[2].imshow(img_hm_bl_1)

    # ax[1].imshow(img_hm_1)
    # ax[2].imshow(img_hm_2)
    # ax[3].imshow(img_hm_3)
    # ax[4].imshow(img_hm_bl_1)
    # ax[5].imshow(img_hm_bl_2)
    # ax[6].imshow(img_hm_bl_3)

plt.show()